<a href="https://colab.research.google.com/github/Zakintaliban/IBM-X-Hactiv8/blob/main/Copy_of_IBM_X_Hacktiv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community
!pip install replicate
!pip install requests pandas numpy python-dotenv requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 1.9 MB/s eta 0:00:00


In [ ]:
from langchain.llms import Replicate
from google.colab import userdata
import os

api_token = userdata.get("api_token")
os.environ["REPLICATE_API_TOKEN"] = api_token

In [ ]:
llm = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    model_kwargs={"max_new_tokens": 10000, "temperature": 0.7, "top_p": 0.9}
)

In [ ]:
# 1. Instalasi Library
import os
import base64
import json
import time
import requests
import pandas as pd
from urllib.parse import urlencode, parse_qs, urlparse
from google.colab import userdata

print("✅ Library berhasil diinstal dan diimpor.")

# 2. Konfigurasi Kredensial
try:
    CLIENT_ID = userdata.get('SPOTIFY_CLIENT_ID')
    CLIENT_SECRET = userdata.get('SPOTIFY_CLIENT_SECRET')
    REDIRECT_URI = "http://127.0.0.1:9090/callback"
    if CLIENT_ID and CLIENT_SECRET:
        print("✅ Konfigurasi dari Colab Secrets berhasil dimuat.")
    else:
        print("❌ Salah satu atau kedua secret kosong.")
except Exception as e:
    print(f"❌ Terjadi error: {e}")

# 3. Otentikasi Spotify
def get_spotify_token(client_id, client_secret, redirect_uri):
    scope = "user-top-read playlist-read-private"
    auth_url = "https://accounts.spotify.com/authorize?" + urlencode({
        "response_type": "code",
        "client_id": client_id,
        "scope": scope,
        "redirect_uri": redirect_uri,
    })
    print("--- OTENTIKASI DIPERLUKAN ---")
    print(f"1. Silakan buka URL berikut di browser Anda:\n\n{auth_url}\n")
    print("2. Setelah login, salin seluruh URL hasil redirect.")
    redirected_url = input("4. Paste URL tersebut di sini: ")

    try:
        parsed_url = urlparse(redirected_url)
        auth_code = parse_qs(parsed_url.query)['code'][0]
    except Exception as e:
        print(f"❌ Gagal parsing URL: {e}")
        return None

    print("🔄 Sedang menukar authorization code dengan access token...")
    auth_options = {
        "url": "https://accounts.spotify.com/api/token",
        "data": {
            "code": auth_code,
            "redirect_uri": redirect_uri,
            "grant_type": "authorization_code"
        },
        "headers": {
            "Authorization": "Basic " + base64.b64encode(f"{client_id}:{client_secret}".encode()).decode(),
            "Content-Type": "application/x-www-form-urlencoded"
        }
    }
    response = requests.post(auth_options['url'], headers=auth_options['headers'], data=auth_options['data'])
    if response.status_code == 200:
        print("✅ Berhasil mendapatkan access token!")
        return response.json()['access_token']
    else:
        print(f"❌ Error: {response.status_code}, {response.text}")
        return None

if 'CLIENT_ID' in locals():
    ACCESS_TOKEN = get_spotify_token(CLIENT_ID, CLIENT_SECRET, REDIRECT_URI)
else:
    print("❌ Kredensial belum dimuat.")

# 4. Fungsi Spotify
def get_user_top_tracks(token, limit=50):
    headers = {"Authorization": f"Bearer {token}"}
    url = f"https://api.spotify.com/v1/me/top/tracks?limit={limit}&time_range=long_term"
    response = requests.get(url, headers=headers)
    return response.json()['items'] if response.status_code == 200 else []

def get_artist_genres(token, artist_ids):
    headers = {"Authorization": f"Bearer {token}"}
    genre_map = {}
    for i in range(0, len(artist_ids), 50):
        chunk = artist_ids[i:i+50]
        ids_str = ",".join(chunk)
        url = f"https://api.spotify.com/v1/artists?ids={ids_str}"
        res = requests.get(url, headers=headers)
        if res.status_code == 200:
            for artist in res.json()['artists']:
                genre_map[artist['id']] = artist['genres']
        time.sleep(0.2)
    return genre_map

def get_user_playlists(token):
    headers = {"Authorization": f"Bearer {token}"}
    playlists = []
    url = "https://api.spotify.com/v1/me/playlists?limit=50"
    while url:
        res = requests.get(url, headers=headers)
        if res.status_code == 200:
            data = res.json()
            playlists += data['items']
            url = data.get('next')
        else:
            break
        time.sleep(0.2)
    return playlists

def get_playlist_tracks(token, playlist_id):
    headers = {"Authorization": f"Bearer {token}"}
    track_ids = set()
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks?fields=items(track(id))&limit=100"
    while url:
        res = requests.get(url, headers=headers)
        if res.status_code == 200:
            for item in res.json()['items']:
                if item.get('track') and item['track'].get('id'):
                    track_ids.add(item['track']['id'])
            url = res.json().get('next')
        else:
            break
        time.sleep(0.2)
    return track_ids

print("✅ Semua fungsi pengambilan data telah didefinisikan.")

# 5. Proses Analisis & AI
if 'ACCESS_TOKEN' in locals() and ACCESS_TOKEN:
    top_tracks_raw = get_user_top_tracks(ACCESS_TOKEN)
    if not top_tracks_raw:
        print("❌ Tidak ada lagu ditemukan.")
    else:
        artist_ids = list(set([t['artists'][0]['id'] for t in top_tracks_raw]))
        genre_map = get_artist_genres(ACCESS_TOKEN, artist_ids)
        user_playlists = get_user_playlists(ACCESS_TOKEN)

        playlist_track_map = {}
        print("\nMengambil lagu dari setiap playlist...")
        for i, pl in enumerate(user_playlists):
            pl_name = ''.join(c for c in pl['name'] if c.isprintable()).strip() or f"Playlist Tanpa Nama {i+1}"
            print(f"  - {pl_name}")
            playlist_track_map[pl_name] = get_playlist_tracks(ACCESS_TOKEN, pl['id'])

        enriched_tracks = []
        for track in top_tracks_raw:
            tid = track['id']
            artist_id = track['artists'][0]['id']
            found_playlists = [pl for pl, ids in playlist_track_map.items() if tid in ids]
            enriched_tracks.append({
                "track_name": track['name'],
                "artist_name": track['artists'][0]['name'],
                "genres": genre_map.get(artist_id, []),
                "moods": found_playlists if found_playlists else ["Tidak Masuk Playlist"]
            })

        df = pd.DataFrame(enriched_tracks)
        print("✅ Data berhasil diproses!\n")

        # Simpan semua print ke summary_for_ai
        from io import StringIO
        import sys
        import numpy as np
        original_stdout = sys.stdout
        temp_stdout = StringIO()
        sys.stdout = temp_stdout

        print("\n\n--- LAPORAN ANALISIS PERSONA MUSIK ANDA ---")
        print("============================================\n")
        all_moods = [mood for mlist in df['moods'] for mood in mlist if mood != "Tidak Masuk Playlist"]
        if not all_moods:
            print("❌ Tidak ada lagu yang terhubung ke playlist.")
        else:
            mood_counts = pd.Series(all_moods).value_counts()
            persona_utama = mood_counts.index[0]
            total = mood_counts.sum()
            print(f"✨ Persona Musik Utama Anda terinspirasi dari Playlist: '{persona_utama}' ✨\n")

            print("\n--- Playlist Mood dan Lagu-Lagu yang Terkait ---")
            for mood in mood_counts.index:
                count = mood_counts[mood]
                percentage = (count / total) * 100
                print(f"\n🎧 '{mood}' muncul {count} kali ({percentage:.1f}%)")
                lagu_mood = df[df['moods'].apply(lambda x: mood in x)]
                for _, row in lagu_mood.iterrows():
                    print(f"  └─ {row['track_name']} | {row['artist_name']}")

            print("\n--- Genre Teratas ---")
            all_genres = [g for g_list in df['genres'] for g in g_list]
            genre_series = pd.Series(all_genres)
            top_genres = genre_series.value_counts().nlargest(10)
            for i, (g, c) in enumerate(top_genres.items(), 1):
                print(f"{i}. {g.title()}")

            print("\n--- 10 Lagu Teratas ---")
            top_songs = [f"{t['name']} - {t['artists'][0]['name']}" for t in top_tracks_raw][:10]
            for i, s in enumerate(top_songs, 1):
                print(f"{i}. {s}")

            print("\n--- 10 Artis Teratas ---")
            top_artists = pd.Series([t['artists'][0]['name'] for t in top_tracks_raw]).value_counts().nlargest(10)
            for i, (a, c) in enumerate(top_artists.items(), 1):
                print(f"{i}. {a}")

            fav_artist = top_artists.idxmax()
            fav_count = top_artists.max()
            print(f"\n--- Artis Favoritmu ---\nKamu paling fanatik sama: {fav_artist} ({fav_count}x)")

            genre_probs = genre_series.value_counts(normalize=True)
            entropy = -np.sum(genre_probs * np.log2(genre_probs))
            print(f"\n--- Genre Diversity ---\nEntropy Skor: {entropy:.3f}")

        # Kembalikan stdout
        sys.stdout = original_stdout
        summary_for_ai = temp_stdout.getvalue()
        print(summary_for_ai)

        # Langkah 6: Kirim ke Granite pakai LangChain LLM
def analyze_music_persona_granular(summary_text):
    print("\n🔍 Menghubungi IBM Granite via LangChain...")
    results = {}

    # Step 2: Granular Prompt Tasks
    prompt_tasks = {
        "persona_insight": """From this summary:\n{summary}\n\nGive a concise **Persona Insight**. Focus on what their music says about their personality, mood, and identity — just the essentials, under 5 points.""",

        "mood_breakdown": """Based on this summary:\n{summary}\n\nSummarize the user's **Mood Breakdown**. Mention the 2–3 strongest emotional tones from their music, with short genre or playlist examples.""",

        "behavior_genre": """Based on this summary:\n{summary}\n\nQuickly analyze their **Genre Behavior**: what do their most repeated genres say about their preferences and diversity? Keep it short.""",

        "behavior_artist_lang": """From this summary:\n{summary}\n\nSummarize their **Artist & Language Behavior** — preferred artists, language variety, and if they like remixes or edits. Max 4 bullet points.""",

        "recommendation_short": """From this summary:\n{summary}\n\nSuggest a **Short-Term Music Recommendation**: specific artists or genres they’d enjoy right now. Limit to 4–5 direct recs.""",

        "recommendation_medium_1": """Based on this summary:\n{summary}\n\nSuggest 3–4 **new genres or subgenres** the user can explore next, related to Emo, K-Pop, or Hardstyle.""",

        "recommendation_medium_2": """From this summary:\n{summary}\n\nGive 3 short **strategies to diversify** their music taste without leaving their comfort zone.""",

        "recommendation_long_1": """From this summary:\n{summary}\n\nSuggest a few **long-term genres or moods** that promote emotional balance and musical growth.""",

        "recommendation_long_2": """Based on this summary:\n{summary}\n\nList 3 **music habits or rituals** that could support their mental/emotional health long-term."""
    }

    # Step 3: Run each task one by one
    import time
    for task_name, prompt_template in prompt_tasks.items():
        prompt_filled = prompt_template.format(summary=summary_text)
        print(f"🔍 Granite - {task_name.replace('_', ' ').title()}")
        start = time.time()
        try:
            response = llm.invoke(prompt_filled)
            duration = time.time() - start
            print(f"✅ Selesai dalam {duration:.2f} detik\n")
            results[task_name] = response
        except Exception as e:
            print(f"❌ Error di task '{task_name}': {e}")
            results[task_name] = None

    # Gabungkan result yang dipecah
    results["recommendation_medium"] = (
        (results.get("recommendation_medium_1") or "") + "\n\n" +
        (results.get("recommendation_medium_2") or "")
    )
    results["recommendation_long"] = (
        (results.get("recommendation_long_1") or "") + "\n\n" +
        (results.get("recommendation_long_2") or "")
    )

    # Bersihkan intermediate
    for key in [
        "recommendation_medium_1", "recommendation_medium_2",
        "recommendation_long_1", "recommendation_long_2"
    ]:
        results.pop(key, None)

    return results

results = analyze_music_persona_granular(summary_for_ai)

# Contoh print hasil:
from IPython.display import Markdown

for key, val in results.items():
    print(f"\n📌 --- {key.replace('_', ' ').title()} ---\n")
    display(Markdown(val))



✅ Library berhasil diinstal dan diimpor.
✅ Konfigurasi dari Colab Secrets berhasil dimuat.
--- OTENTIKASI DIPERLUKAN ---
1. Silakan buka URL berikut di browser Anda:

https://accounts.spotify.com/authorize?response_type=code&client_id=5bea2392267d4e7d8fa7cf2171294711&scope=user-top-read+playlist-read-private&redirect_uri=http%3A%2F%2F127.0.0.1%3A9090%2Fcallback

2. Setelah login, salin seluruh URL hasil redirect.
4. Paste URL tersebut di sini: http://127.0.0.1:9090/callback?code=AQCqGOsfMoSosuHpgiHKSUNgCExvZ8kQykVblX1w_YI5Di0tYMzQ95dVtYjugBNyAb2lvjvZWKLxASkjDAK1vEuygblsjjFFkVF8mpj6AAw3TXFW3--Kss6RbYcP5Mj04C_vmOG7EtL_dNtMJAA3rZqyZz3V1sAXnhHp27D60GOoBZl74ojlaQja-R7W855DXMnrkF9uJoFKvq-HNs5Y4jS4dXM_KukU
🔄 Sedang menukar authorization code dengan access token...
✅ Berhasil mendapatkan access token!
✅ Semua fungsi pengambilan data telah didefinisikan.

Mengambil lagu dari setiap playlist...
  - rekomendasi
  - rekomendasi
  - Chill
  - 🖕🖕🖕
  - 杏里シティポップ～ANRI's selected CITY POP MASTERPIECE!
 

1. This individual identifies with Emo and Pop Punk genres, suggesting an affinity for introspective, often melancholic lyrics and energetic, guitar-driven music.

2. Their playlist dominance indicates a preference for curated playlists, possibly reflecting a methodical or organized personality. The prevalence of playlists with personal themes ('This playlist is literally me', 'My 2024 Playlist in a Bottle') suggests a desire to express individuality and personal identity through music.

3. The inclusion of Hardstyle, Indonesian Pop, Malay, and K-Pop genres, alongside specific artist preferences like My Chemical Romance and LE SSERAFIM, points to a diverse musical palate and an openness to various cultural influences.

4. The top songs list reveals a balance between high-energy dance tracks (Hardstyle, Jersey Club) and emotionally resonant ballads (My Chemical Romance), hinting at a multifaceted personality that can appreciate both high-octane excitement and introspective moments.

5. The presence of less common genres like Phonk, Drift Phonk, and unique remixes (like the sped-up K-Pop track) suggests an adventurous spirit and a keen interest in niche music subcultures.


📌 --- Mood Breakdown ---



Based on the provided summary, the user's mood breakdown reveals the following three strongest emotional tones in their music:

1. **Emo and Pop Punk:** These genres are evident in the top 10 songs, with "I'm Not Okay (I Promise)" by My Chemical Romance appearing prominently. Playlists like '🤫🧏♂️🎧🏋️♀️' and 'My 2024 Playlist in a Bottle' also indicate an inclination towards emotional, introspective music from these genres.

2. **Energetic and Dance-Oriented:** The presence of Hardstyle, Jersey Club, Phonk, and Drift Phonk in the dominant genres, along with songs like 'OUTSIDE HARDSTYLE - David Laid Edition - Daciva' and 'bounce (i just wanna dance) (sped up) - фрози', suggests an appreciation for high-energy, dance-oriented tracks that might reflect feelings of excitement, confidence, or even catharsis.

3. **Indonesian and Malay Pop:** With playlists like 'Martabak Spesial' and 'Pak Vramroro Fufufafa - Aestheards', the user seems to enjoy Indonesian and Malay pop music. These songs may evoke feelings of nostalgia, cultural connection, or simply joy associated with familiar, upbeat tunes.

In summary, the user's mood breakdown leans towards Emo/Pop Punk for introspective, emotional expression; Energetic/Dance-Oriented for high-spirited, lively experiences; and Indonesian/Malay Pop for cultural affinity and cheerful vibes.


📌 --- Behavior Genre ---



...

Based on the provided summary, here are four short-term music recommendations focusing on the dominant genres and artists:

1. **Emo/Pop Punk:** Given the prominence of Emo and Pop Punk, users might enjoy more music from similar artists. Recommendations include:
   - *Fall Out Boy*
   - *Paramore*
   - *The Used*
   - *Panic! at the Disco*

2. **Indonesian Pop:** The summary highlights Indonesian Pop as one of the main genres. For users interested in exploring this style further:
   - *Tiara Andini* (already in the Top 10 songs)
   - *Astrid* (popular Indonesian singer)
   - *Maia Estianty* (well-known Indonesian pop artist)

3. **K-Pop:** K-Pop is listed as one of the main genres. If users enjoy the provided Top 10 songs, they might like:
   - *BTS*
   - *Blackpink*
   - *Twice*
   - *Exo*

4. **Phonk/Drift Phonk:** These genres are also featured prominently. For those interested in exploring these styles:
   - *Sublime with Rome* (known for blending Phonk with other genres)
   - *Tidal Waves (feat. TeeJay & Kofi) - Phonk Remix* by various artists
   - *Drift Phonk* playlists on music streaming platforms (e.g., Spotify)

These recommendations should align with the dominant genres and artists in the user's playlist, providing a cohesive and enjoyable short-term listening experience.


📌 --- Recommendation Medium ---



Based on the provided summary, the user seems to have a preference for Emo, K-Pop, and Hardstyle genres. Here are four new genres or subgenres they could explore next, keeping in close relation to their interests:

1. **Emo Revival:** This subgenre emerged around the 2010s, blending classic emo with elements of indie rock, pop-punk, and post-hardcore. Bands like The Hotelier, Modern Baseball, and The Sidekicks exemplify this style.

2. **K-Pop's Indie-Influenced Subgenre:** Explore the indie-influenced side of K-Pop, which incorporates elements of alternative rock, folk, and dream pop. Artists such as Hyukoh, Epik High, and Candlebox (a Korean band with American influences) fall under this category.

3. **Hardstyle Subgenres:** The user might enjoy exploring more nuanced subgenres within Hardstyle, such as:
   - **Raw Hardstyle:** Characterized by its raw, heavy kicks and minimalistic sound, pioneered by artists like Headhunterz, Noisia, and The Prophet.
   - **Future Hardstyle:** A more melodic and progressive subgenre, incorporating elements of trance and electro. Notable artists include Denny Milla, Brennan Heart, and D-Block & Sruv.

4. **Post-Hardcore/Screamo:** This genre is closely related to Emo and often features intense, aggressive vocals. Explore bands like Thrice, La Dispute, and Pianos Become the Teeth for a deeper dive into this subgenre.

These suggestions should help the user expand their musical horizons while staying within their preferred genres and subgenres.

Based on the given summary, the user's music taste leans heavily towards Emo, Pop Punk, and specific genres like Hardstyle, Indonesian Pop, and Malay music. Here are three strategies to diversify their music taste while staying within a comfortable range:

1. **Explore Subgenres:** Encourage the user to delve deeper into related subgenres that they haven't extensively listened to yet. For instance, since they enjoy Emo and Pop Punk, they might appreciate Post-Hardcore, Emo Revival, or Indie Rock. They could also explore different waves of Hardstyle, such as Early, Raw, or Tech Hardstyle, which still retain energy but have distinct sounds.

2. **Global Artists within Preferred Genres:** Introduce them to international artists within their favored genres. For example, in the Emo/Pop Pck scene, they might enjoy bands like 'The Story So Far' (USA), 'Neck Deep' (UK), or 'State Champs' (USA). In the Indonesian and Malay Pop scene, they could discover more artists like Maia, Ariel, or Nidji. This will broaden their horizons without straying too far from familiar ground.

3. **Genre Fusion Playlists:** Suggest playlists that blend their preferred genres with others. For example, they might enjoy Emo-influenced K-Pop (like 'Stray Kids' or 'Texter') or Indonesian Pop infused with Electronic Dance Music (EDM) elements. This way, they can experience new combinations of sounds they already appreciate. Online platforms like Spotify often curate such genre-blending playlists, making discovery easier.

These strategies aim to introduce variety gradually, ensuring the user remains comfortable while exploring new musical territories.


📌 --- Recommendation Long ---



Based on the provided summary, the dominant genres and moods in this playlist are predominantly centered around Emo, Pop Punk, Hardstyle, and Indonesian/Malay Pop, with a sprinkling of K-Pop, Phonk, Jersey Club, EDM, and Drift Phonk. To suggest long-term genres or moods that promote emotional balance and musical growth, we should consider genres that encourage exploration, reflection, and a broad spectrum of emotions. Here are a few recommendations:

1. **Alternative/Indie Rock**: This genre often combines poetic lyrics with diverse musical arrangements, offering a wide range of emotions. Artists like Bon Iver, The National, and Fleet Foxes can provide a balanced blend of introspective and uplifting vibes.

2. **Classical Music**: Exploring the vast repertoire of classical compositions can enhance one's understanding of musical structure, harmony, and emotional expression. Pieces by composers such as Mozart, Beethoven, or contemporary classical artists like Ólafur Arnalds can provide a calming yet intellectually stimulating experience.

3. **Ambient/Electronic**: Ambient and electronic music, particularly subgenres like downtempo, chillout, or even some IDM, can promote relaxation and introspection. Artists like Brian Eno, Tycho, or Boards of Canada create immersive soundscapes that allow listeners to reflect and unwind.

4. **World Music**: Expanding the musical horizons by exploring diverse world music traditions, such as African, Latin American, or Middle Eastern genres, can provide fresh perspectives and emotional depth. Artists like Angélique Kidjo, Buena Vista Social Club, or Oumou Sangaré can offer a rich tapestry of rhythms and melodies.

5. **Jazz**: Jazz, with its emphasis on improvisation and emotional expression, can help develop a deeper appreciation for musicianship and emotional range. Artists like Miles Davis, John Coltrane, or Ella Fitzgerald can provide a blend of soph

Based on the provided summary, here are three music habits or rituals that could support the user's mental and emotional health long-term:

1. **Curated Playlists for Self-Expression and Mood Management:**
   The user seems to engage with playlists that resonate with their personal experiences and emotions, such as '🤫🧏♂️🎧🏋️♀️' (17.1%) and 'This playlist is literally me' (11.4%). Creating and regularly updating playlists tailored to specific moods or life phases can be a powerful tool for emotional expression and regulation. This habit allows the user to process feelings, find solace in relatable music, and foster a sense of identity and self-understanding.

2. **Exploring Diverse Genres and Artists:**
   The playlist and top artist list showcases a variety of genres, including Emo, Pop Punk, Indonesian Pop, Malay, K-Pop, Phonk, Jersey Club, Edm, and Drift Phonk. Engaging with a wide range of musical styles can contribute to mental well-being by exposing the user to different perspectives, cultures, and emotions. This diversity can foster open-mindedness, reduce stress, and promote a sense of connection to the broader human experience.

3. **Incorporating Calming and Energizing Music:**
   The presence of 'Chill' (5.7%) and the top songs like 'Tip Toe' (HYBS) and 'Perfect Night' (LE SSERAFIM) suggests that the user seeks music for relaxation and upliftment. Intentionally integrating soothing and energizing tracks into their listening routine can help manage stress, boost mood, and improve overall mental well-being. Regularly alternating between calming and energizing music can create a balanced emotional state, promoting resilience and emotional stability in the long term.

By consistently engaging in these music habits, the user can nurture their mental and emotional health, fostering self-awareness, adaptability, and a positive outlook on life.